In [60]:
import os
import PyPDF2 as pypdf
#import pdfminer
import pandas as pd
import re

#data_dir = '../data'
data_dir = '../Albuquerque3/'

In [132]:
def get_court(text):
    pypdf2_text.split("\n")[1]
    return court

def get_acts(text):
    """
    takes the text of a court case pdf and returns a dictionary of the
    acts (prostitution, pimping, pandering, commercial sex act, sex trafficking) and their
    occurence numbers in the court document
    """
    return acts_dict

# takes the text of a court case pdf and returns wh
def get_elements(text):
    """
    takes the text of a court case pdf and returns a dictionary of the
    elements (force, fraud, coercion, commercial sex act) and their
    occurence numbers in the court document organized by ocurrence descending
    """
    return df

# takes the text of a court case pdf and returns which court heard the case
def get_arresting_dept(text):
    """
    takes the text of a court case pdf and returns a dictionary of the
    possible arresting agencies (agency:ocurrence), to be used in determining arresting agency
    for affidavit requests
    """
    return df

def get_courtcode(text, code):
    return text.count(code)

def get_alb_pd(text):
    return text.count("albuquerque police") + text.count("albuquerque pd") 

def get_all_pd(text):
    return text.count("pd") + text.count("police dep") 

def get_defendant_name(text):
    
    return name

def get_trafficking_relevant(text):
    pimp_count = text.count("pimp") + text.count("daddy") + text.count("boss")
    advertise_count = text.count('advertise')
    drug_count = text.count("drug")
    hotel_motel_count = text.count("hotel") + text.count("motel") + text.count("room")
    money_count = text.count("money") + text.count("cash") + text.count("paid")
    trafficking_count = text.count("trafficking")
    prostitue_count = text.count("prosititute") + text.count("prositution") + text.count("bitch") + text.count("escort")
    travel_count = text.count("travel")
    sex_count = text.count("sex")
    force_count = text.count("force")
    fraud_count = text.count("fraud")
    coercion_count = text.count("coercion")
    relevant_elements_count = pimp_count + advertise_count + trafficking_count + hotel_motel_count
    + money_count + prostitue_count + travel_count + sex_count
    if (relevant_elements_count > 0):
        if ((trafficking_count > 0) and (sex_count > 0)):
            if ((prostitue_count > 0) or (pimp_count > 0)):
                    trafficking_relevance = 'Very Likely Relevant'
            else:
                trafficking_relevance = 'Likely Relevant'
        elif (trafficking_count > 0):
            if ((pimp_count == 0) and (drug_count > 0)):
                trafficking_relevance = 'Likely Not Relevant'
            elif ((prostitue_count > 0) or (pimp_count > 0)):
                trafficking_relevance = 'Likely Relevant'
            else:
                trafficking_relevance = 'Likely Not Relevant'
        elif (trafficking_count == 0):
            if (pimp_count > 0):
                trafficking_relevance = 'Likely Relevant'
            elif (prostitue_count > 0):
                trafficking_relevance = 'Maybe Relevant'
            else:
                trafficking_relevance = 'Likely Not Relevant'
        else:
            trafficking_relevance = 'Likely Not Relevant'
            
    else:
        trafficking_relevance = 'Very Likely Not Relevant'
    return relevant_elements_count, trafficking_relevance

def get_other_crimes(text):
    core_terms = ' '.join(text.partition("core terms\n")[2].split('\n')[0:6])
    rape = text.count("rape")
    murder = text.count("murder") + text.count("kill") + text.count("homicide")
    kidnapping = text.count("kidnap") + text.count("abduct")
    money_laundering = text.count("laundering")
    
    if (rape > 0):
        rape_flag = "Yes"
    else:
        rape_flag = "No"
        
    if (murder > 0):
        murder_flag = "Yes"
    else:
        murder_flag = "No"
        
    if (kidnapping > 0):
        kidnapping_flag = "Yes"
    else:
        kidnapping_flag = "No"
        
    if (money_laundering > 0):
        money_laundering_flag = "Yes"
    else:
        money_laundering_flag = "No"
        
    
    return rape_flag, murder_flag, kidnapping_flag, money_laundering_flag
    
"""
def get_1591(text): 
    if "1591" in text:
        occurences = text.count("1591")
    else:
        occurences = 0
    return occurences

def get_2421(text): 
    if "2421" in text:
        occurences = text.count("2421")
    else:
        occurences = 0
    return occurences

def get_1594(text): 
    if "1594" in text:
        occurences = text.count("1594")
    else:
        occurences = 0
    return occurences

def get_2422(text): 
    if "2422" in text:
        occurences = text.count("2422")
    else:
        occurences = 0
    return occurences

def get_2423(text): 
    if "2423" in text:
        occurences = text.count("2423")
    else:
        occurences = 0
    return occurences

def get_1328(text): 
    if "1328" in text:
        occurences = text.count("1328")
    else:
        occurences = 0
    return occurences

def get_1952(text): 
    if "1952" in text:
        occurences = text.count("1952")
    else:
        occurences = 0
    return occurences

def get_word(text, word):
    if word.str() in text: 
        occurences = text.count(word.str())
    else:
        occurences = 0 
    return occurences 
    
"""

print("")

In [141]:
files = []
c1591 = []
c2421 = []
c1594 = []
c2422 = []
c2423 = []
c1328 = []
c1952 = []
trafficking_elements = []
trafficking_relevance = []
rape = []
murder = []
kidnapping = []
money_laundering = []
violent_crime = []
assault = []
alb_pd = []
all_pd = []

for filename in os.scandir(data_dir):
        if ".pdf" in filename.name:
            reader = pypdf.PdfReader(filename.path)
            pypdf2_text = ""
            for i in range(len(reader.pages)):
                pypdf2_text += reader.pages[i].extract_text()  
            pypdf2_text = pypdf2_text.lower()
            print(filename.name)
            files.append(filename.name)
            c1591.append(get_courtcode(pypdf2_text, "1591"))
            c2421.append(get_courtcode(pypdf2_text, "2421"))
            #c1594.append(get_courtcode(pypdf2_text, "1594"))
            #c2422.append(get_courtcode(pypdf2_text, "2422"))
            #c2423.append(get_courtcode(pypdf2_text, "2423"))
            #c1328.append(get_courtcode(pypdf2_text, "1328"))
            #c1952.append(get_courtcode(pypdf2_text, "1952"))
            elements_count,trafficking_relevant = get_trafficking_relevant(pypdf2_text)
            trafficking_elements.append(elements_count)
            trafficking_relevance.append(trafficking_relevant)
            rape_flag, murder_flag, kidnapping_flag, money_laundering_flag = get_other_crimes(pypdf2_text)
            rape.append(rape_flag)
            murder.append(murder_flag)
            kidnapping.append(kidnapping_flag)
            money_laundering.append(money_laundering_flag)
            violent_crime.append(violent_crime_flag)
            assault.append(assault_flag)
            alb_pd.append(get_alb_pd(pypdf2_text))
            all_pd.append(get_all_pd(pypdf2_text))
        else:
            continue

A.C. v. C.B., 113 N.M. 581.pdf
A.M. v. N.M. Dep_t of Health, 108 F. Supp. 3d 963.pdf
A.M. v. N.M. Dep_t of Health, 148 F. Supp. 3d 1232.pdf
Abeyta by _ Through Martinez v. Chama Valley Indep. Sch. Dist. No. 19, 77 F.3d 1253.pdf
ACLU of N.M. v. City of Albuquerque, 139 N.M. 761.pdf
ACLU v. Johnson, 4 F. Supp. 2d 1024.pdf
Aguilar v. Aramark Corp., 1998 U.S. Dist. LEXIS 24654.pdf
Aguilar v. Las Cumbres Learning Servs., 2008 U.S. Dist. LEXIS 69080.pdf
Aguilar v. McAleenan, 2019 U.S. Dist. LEXIS 194935.pdf
Allen v. City of Albuquerque, 2015 U.S. Dist. LEXIS 198991.pdf
Allen v. United States, 588 F. Supp. 247.pdf
Alvarado v. Donley, 2011 U.S. Dist. LEXIS 14436.pdf
Alvarado v. KOB-TV, L.L.C., 493 F.3d 1210.pdf
Alvey v. Janecka, 2007 U.S. Dist. LEXIS 114505.pdf
American Booksellers Asso. v. Schiff, 868 F.2d 1199.pdf
Anaya v. CBS Broad. Inc., 626 F. Supp. 2d 1158.pdf
Andazola v. County of Chaves, 2001 U.S. Dist. LEXIS 28254.pdf
Apodaca v. Mazer, 1998 U.S. Dist. LEXIS 24696.pdf
Apodaca v. Public

Douglass v. Mut. Benefit Health _ Accident Ass_n, 42 N.M. 190.pdf
Drevaleva v. United States VA, 2022 U.S. App. LEXIS 18957.pdf
Dumais v. Am. Golf Corp., 2003 U.S. Dist. LEXIS 30172.pdf
Dummar v. Lummis, 543 F.3d 614.pdf
Dunn v. Smith, 2022 U.S. Dist. LEXIS 50991.pdf
Duprey v. Twelfth Judicial Dist. Court, 2009 U.S. Dist. LEXIS 71113.pdf
Duprey v. Twelfth Judicial Dist. Court, 760 F. Supp. 2d 1180.pdf
Duran v. N.M. Dep_t of Human Servs., Income Support Div., 95 N.M. 196.pdf
E.M. v. Espanola, 2008 U.S. Dist. LEXIS 144338.pdf
Eaton v. Bernalillo County, 46 N.M. 318.pdf
Eckert v. City of Deming, 2015 U.S. Dist. LEXIS 176612.pdf
Edwell v. Chase, 2005 U.S. Dist. LEXIS 39269.pdf
EEOC v. Arg Enter., 2009 U.S. Dist. LEXIS 147954.pdf
EEOC v. BCI Coca-Cola Bottling Co., 2004 U.S. Dist. LEXIS 28277.pdf
EEOC v. JBS USA, LLC, 339 F. Supp. 3d 1135.pdf
EEOC v. Mead Foods, Inc., 466 F. Supp. 1.pdf
EEOC v. Sandia Corp., 639 F.2d 600.pdf
EEOC v. Sky King, Inc., 1982 U.S. Dist. LEXIS 15479.pdf
EEOC v. Un

Kvech v. N.M. Dep_t of Public Safety, 987 F. Supp. 2d 1162.pdf
L.W. v. Gallup McKinley County Sch. Bd., 2002 U.S. Dist. LEXIS 30319.pdf
La Casa de Buena Salud v. United States, 2008 U.S. Dist. LEXIS 42352.pdf
Laul v. Los Alamos Nat_l Labs., 309 F. Supp. 3d 1119.pdf
Laurich v. Red Lobster Rests., LLC, 295 F. Supp. 3d 1186.pdf
Leal v. Falk, 2020 U.S. Dist. LEXIS 252494.pdf
Lee v. Univ. of N.M., 449 F. Supp. 3d 1071.pdf
Lee v. Univ. of New Mexico, 500 F. Supp. 3d 1181.pdf
Legacy Church, Inc. v. Kunkel, 455 F. Supp. 3d 1100.pdf
Legacy Church, Inc. v. Kunkel, 472 F. Supp. 3d 926.pdf
Leszinske v. Poole, 110 N.M. 663.pdf
Lewis v. D. R. Horton, Inc., 2008 U.S. Dist. LEXIS 130583.pdf
Lewis v. Sandoval, 2009 U.S. Dist. LEXIS 146300.pdf
Leyba v. City of Santa Fe, 2017 U.S. Dist. LEXIS 21343.pdf
Le_Mon v. NLRB, 902 F.2d 810.pdf
Lin v. Peters, 2001 U.S. Dist. LEXIS 28316.pdf
Lindenau v. Alexander, 663 F.2d 68.pdf
Littell v. Allstate Ins. Co., 2003 U.S. Dist. LEXIS 29174.pdf
Littlefield v. Piedra Vi

Ramsay v. Frontier, Inc., 2020 U.S. Dist. LEXIS 151176.pdf
Rangel v. El Paso Natural Gas Co., 996 F. Supp. 1093.pdf
Rawers v. United States, 2020 U.S. Dist. LEXIS 174622.pdf
Rawers v. United States, 545 F. Supp. 3d 1087.pdf
Ray v. Astrue, 2008 U.S. Dist. LEXIS 138407.pdf
Rayos v. State ex rel. Dep_t of Corrs., 2014-NMCA-103.pdf
Reid v. Pautler, 36 F. Supp. 3d 1067.pdf
Renaud v. Eyemart Express, 2002 U.S. Dist. LEXIS 30254.pdf
Riddle v. Mondragon, 83 F.3d 1197.pdf
Rivas v. Apfel, 1999 U.S. Dist. LEXIS 24657.pdf
Rivero v. Bd. of Regents of the Univ. of N.M., 2019 U.S. Dist. LEXIS 36803.pdf
Robertson v. USDA, 2003 U.S. Dist. LEXIS 29932.pdf
Robinson v. State, 34 N.M. 557.pdf
Rodriguez v. New Mexico Children Youth _ Families Dep_t, 2004 U.S. Dist. LEXIS 34624.pdf
Romero v. Bradford, 2010 U.S. Dist. LEXIS 164477.pdf
Romero v. Gonzales (In re Gonzales), 2007 Bankr. LEXIS 1438.pdf
Romero v. Mason _ Hanger-Silas Mason Co., 739 F. Supp. 1472.pdf
Rosen v. U.S. Bank Nat_l Ass_n, 2006 U.S. Dist. L

State v. Jones, 120 N.M. 185.pdf
State v. Kenneth, 2015 N.M. App. Unpub. LEXIS 408.pdf
State v. Kerby, 138 N.M. 232.pdf
State v. Ketchum, 2013 N.M. App. Unpub. LEXIS 91.pdf
State v. Keyonnie, 91 N.M. 146.pdf
State v. Kleinegger, 2021 N.M. App. Unpub. LEXIS 163.pdf
State v. Lamure, 115 N.M. 61.pdf
State v. Larson, 94 N.M. 795.pdf
State v. Leathers, 2011 N.M. App. Unpub. LEXIS 309.pdf
State v. Leeson, 149 N.M. 823.pdf
State v. Leiding, 112 N.M. 143.pdf
State v. Lente, 138 N.M. 312.pdf
State v. Lewis, 116 N.M. 849.pdf
State v. Lewis, 2020 N.M. App. Unpub. LEXIS 195.pdf
State v. Lopez, 142 N.M. 613.pdf
State v. Lovato, 2011 N.M. App. Unpub. LEXIS 331.pdf
State v. Lovett, 2012-NMSC-036.pdf
State v. Lowe, 135 N.M. 520.pdf
State v. Loza, 2018-NMSC-034.pdf
State v. Lucero, 118 N.M. 696.pdf
State v. Lucero, 2019 N.M. App. Unpub. LEXIS 416.pdf
State v. Malloy, 131 N.M. 222.pdf
State v. Mann, 131 N.M. 459.pdf
State v. Manzanares, 2011 N.M. App. Unpub. LEXIS 18.pdf
State v. Marquez, 2016-NMSC-025.

United States v. Bautista, 145 F.3d 1140.pdf
United States v. Bearden, 2015 U.S. Dist. LEXIS 196615.pdf
United States v. Beers, 110 Fed. Appx. 53.pdf
United States v. Beers, 189 F.3d 1297.pdf
United States v. Begay, 2008 U.S. Dist. LEXIS 145260.pdf
United States v. Begay, 497 F. Supp. 3d 1025.pdf
United States v. Begay, 550 Fed. Appx. 604.pdf
United States v. Beltran-Rivera, 2010 U.S. Dist. LEXIS 63227.pdf
United States v. Beltran-Rivera, 670 F. Supp. 2d 1207.pdf
United States v. Bhula, 2020 U.S. Dist. LEXIS 224406.pdf
United States v. Bhula, 2021 U.S. Dist. LEXIS 183076.pdf
United States v. Bhula, 2022 U.S. Dist. LEXIS 225203.pdf
United States v. Bhula, 2022 U.S. Dist. LEXIS 233394.pdf
United States v. Bhula, 2023 U.S. Dist. LEXIS 25241.pdf
United States v. Bhula, 2023 U.S. Dist. LEXIS 2826.pdf
United States v. Bhula, 2023 U.S. Dist. LEXIS 4641.pdf
United States v. Bitsilly, 386 F. Supp. 2d 1192.pdf
United States v. Brown, 784 F.2d 1033.pdf
United States v. Bruce, 458 F.3d 1157.pdf
Un

United States v. Hernandez-Castillo, 449 F.3d 1127.pdf
United States v. Hernandez-Hernandez, 519 F.3d 1236.pdf
United States v. Herrera, 286 Fed. Appx. 546.pdf
United States v. Heywood, 2020 U.S. Dist. LEXIS 150620.pdf
United States v. Holguin-Chavez, 279 Fed. Appx. 668.pdf
United States v. Hopkins, 927 F. Supp. 2d 1120.pdf
United States v. Hurtado, 2017 U.S. Dist. LEXIS 53131.pdf
United States v. Jack, 2014 U.S. Dist. LEXIS 185058.pdf
United States v. Jackson, 2018 U.S. Dist. LEXIS 211942.pdf
United States v. Jager, 2011 U.S. Dist. LEXIS 21203.pdf
United States v. Jim, 2011 U.S. Dist. LEXIS 138810.pdf
United States v. Jim, 786 F.3d 802.pdf
United States v. Jim, 839 F. Supp. 2d 1157.pdf
United States v. Jim, 877 F. Supp. 2d 1018.pdf
United States v. Joe, 8 F.3d 1488.pdf
United States v. Johnson, 183 F.3d 1175.pdf
United States v. Johnson, 2021 U.S. Dist. LEXIS 1794.pdf
United States v. Johnson, 364 F.3d 1185.pdf
United States v. Jojola, 608 F. Supp. 3d 1050.pdf
United States v. Jones, 

United States v. Wallen, 2005 U.S. Dist. LEXIS 60904.pdf
United States v. Waseta, 647 F.3d 980.pdf
United States v. Williams, 2020 U.S. Dist. LEXIS 124956.pdf
United States v. Willie, 253 F.3d 1215.pdf
United States v. Wolfe, 435 F.3d 1289.pdf
United States v. Woods, 2020 U.S. Dist. LEXIS 150676.pdf
United States v. Woods, 2020 U.S. Dist. LEXIS 207907.pdf
United States v. Woods, 2021 U.S. Dist. LEXIS 76072.pdf
United States v. Woods, 2021 U.S. Dist. LEXIS 81858.pdf
United States v. Woody, 2021 U.S. Dist. LEXIS 12110.pdf
United States v. Woody, 336 F.R.D. 293.pdf
United States v. Yanez-Rodriguez, 555 F.3d 931.pdf
United States v. Yates, 22 F.3d 981.pdf
United States v. Yazzie, 1998 U.S. App. LEXIS 10485.pdf
United States v. Yazzie, 2013 U.S. Dist. LEXIS 90265.pdf
United States v. Yazzie, 2014 U.S. Dist. LEXIS 66729.pdf
United States v. Yazzie, 2014 U.S. Dist. LEXIS 66730.pdf
United States v. Yazzie, 660 F.2d 422.pdf
United States v. Yazzie, 998 F. Supp. 2d 1044.pdf
United States v. Yepa

In [142]:
len(files), len(c1952)

(1158, 0)

In [143]:
cases = pd.DataFrame({'file_name': files,
                      'Code_1591': c1591,
                      'Code_2421': c2421, 
                      #'Code_1594': c1594, 
                      #'Code_2422': c2422, 
                      #'Code_2423': c2423, 
                      #'Code_1328': c1328, 
                      #'Code_1952': c1952, 
                      'Trafficking_Elements': trafficking_elements,
                      'Trafficking_Relevance': trafficking_relevance,
                      'AlbPD_Mentions':alb_pd, 
                      'AllPD_Mentions': all_pd,
                      'Rape':rape,
                      'Murder':murder,
                      'Kidnapping':kidnapping,
                      'Money_Laundering':money_laundering,
                      'Violent_Crime':violent_crime,
                      'Assault':assault})
len(cases[cases['AlbPD_Mentions']>0])

123

In [144]:
cases.head()

,file_name,Code_1591,Code_2421,Trafficking_Elements,Trafficking_Relevance,AlbPD_Mentions,AllPD_Mentions,Rape,Murder,Kidnapping,Money_Laundering,Violent_Crime,Assault
0,"A.C. v. C.B., 113 N.M. 581.pdf",0,0,0,Very Likely Not Relevant,0,0,No,No,No,No,No,Yes
1,"A.M. v. N.M. Dep_t of Health, 108 F. Supp. 3d ...",0,0,10,Maybe Relevant,0,0,Yes,Yes,No,No,No,No
2,"A.M. v. N.M. Dep_t of Health, 148 F. Supp. 3d ...",0,0,2,Maybe Relevant,0,0,Yes,Yes,No,No,No,No
3,Abeyta by _ Through Martinez v. Chama Valley I...,0,0,0,Very Likely Not Relevant,0,0,No,Yes,No,No,No,Yes
4,"ACLU of N.M. v. City of Albuquerque, 139 N.M. ...",0,0,1,Likely Not Relevant,1,11,No,No,Yes,No,No,No


In [146]:
code_cols = cases.columns[1:2]

In [147]:
cases['TotalCharges'] = cases[code_cols].sum(axis = 1)

In [148]:
cases.head()

,file_name,Code_1591,Code_2421,Trafficking_Elements,Trafficking_Relevance,AlbPD_Mentions,AllPD_Mentions,Rape,Murder,Kidnapping,Money_Laundering,Violent_Crime,Assault,TotalCharges
0,"A.C. v. C.B., 113 N.M. 581.pdf",0,0,0,Very Likely Not Relevant,0,0,No,No,No,No,No,Yes,0
1,"A.M. v. N.M. Dep_t of Health, 108 F. Supp. 3d ...",0,0,10,Maybe Relevant,0,0,Yes,Yes,No,No,No,No,0
2,"A.M. v. N.M. Dep_t of Health, 148 F. Supp. 3d ...",0,0,2,Maybe Relevant,0,0,Yes,Yes,No,No,No,No,0
3,Abeyta by _ Through Martinez v. Chama Valley I...,0,0,0,Very Likely Not Relevant,0,0,No,Yes,No,No,No,Yes,0
4,"ACLU of N.M. v. City of Albuquerque, 139 N.M. ...",0,0,1,Likely Not Relevant,1,11,No,No,Yes,No,No,No,0


In [150]:
len(cases)

1158

In [151]:
cases.to_csv('../Albuquerque/FilteredAlbuquerque.csv')

# Seth working on names

In [ ]:
'''
files = []
c1591 = []
c2421 = []
c1594 = []
c2422 = []
c2423 = []
c1328 = []
c1952 = []
alb_pd = []
all_pd = []
'''
for filename in os.scandir(data_dir):
        if ".pdf" in filename.name:
            reader = pypdf.PdfReader(filename.path)
            pypdf2_text = ""
            for i in range(len(reader.pages)):
                pypdf2_text += reader.pages[i].extract_text()  
            files.append(filename.name)
            
            '''
            c1591.append(get_courtcode(pypdf2_text, "1591"))
            c2421.append(get_courtcode(pypdf2_text, "2421"))
            c1594.append(get_courtcode(pypdf2_text, "1594"))
            c2422.append(get_courtcode(pypdf2_text, "2422"))
            c2423.append(get_courtcode(pypdf2_text, "2423"))
            c1328.append(get_courtcode(pypdf2_text, "1328"))
            c1952.append(get_courtcode(pypdf2_text, "1952"))
            alb_pd.append(get_alb_pd(pypdf2_text))
            all_pd.append(get_all_pd(pypdf2_text))
            '''
        else:
            continue

In [ ]:
cases = pd.DataFrame({'file_name': files,
                      'Code_1591': c1591,
                      'Code_2421': c2421, 
                      'Code_1594': c1594, 
                      'Code_2422': c2422, 
                      'Code_2423': c2423, 
                      'Code_1328': c1328, 
                      'Code_1952': c1952, 
                     'AlbPD_Mentions':alb_pd, 
                     'AllPD_Mentions': all_pd})
cases

# break
-------------------------------------------------------------

In [30]:
cases

,file_name,Code_1591,Code_2421,Code_1594,Code_2422,Code_2423,Code_1328,Code_1952,AlbPD_Mentions,AllPD_Mentions,TotalCharges
0,"A.M. v. N.M. Dep_t of Health, 148 F. Supp. 3d ...",0,0,0,0,0,0,0,0,0,0
1,"ACLU of N.M. v. City of Albuquerque, 139 N.M. ...",0,0,0,0,0,0,0,1,12,0
2,"Aguilar v. McAleenan, 2019 U.S. Dist. LEXIS 19...",0,0,0,0,0,0,0,0,0,0
3,"Allen v. City of Albuquerque, 2015 U.S. Dist. ...",0,0,0,0,0,0,0,0,3,0
4,"Alvarado v. KOB-TV, L.L.C., 493 F.3d 1210.pdf",0,0,0,0,0,0,0,3,6,0
...,...,...,...,...,...,...,...,...,...,...,...
266,"Webb v. Padilla, 2009 U.S. Dist. LEXIS 101489.pdf",0,0,0,0,0,0,0,3,8,0
267,"White v. Town of Hurley, 2019 U.S. Dist. LEXIS...",0,0,0,0,0,0,0,0,39,0
268,"Womack v. Unified Gov_t, 2021 U.S. Dist. LEXIS...",0,0,0,0,0,0,0,0,17,0
269,"Young v. Wilham, 2017-NMCA-087.pdf",0,0,0,0,0,1,0,1,42,1


In [31]:
# Read in the text from case PDFs and store in dataframe
cases = pd.DataFrame({'file_name': pd.Series(dtype='str'),
                      #'court': pd.Series(dtype='str'),                      
                      #'acts': pd.Series(dtype='str'),
                      #'elements_used': pd.Series(dtype='str'),
                      #'location_likely': pd.Series(dtype='str'),
                      #'arresting_department_likely': pd.Series(dtype='float'), 
                      'Code_1591': pd.Series(dtype='int'), 
                      'Code_2421': pd.Series(dtype='int'), 
                      'Code_1594': pd.Series(dtype='int'), 
                      'Code_2422': pd.Series(dtype='int'), 
                      'Code_2423': pd.Series(dtype='int'), 
                      'Code_1328': pd.Series(dtype='int'), 
                      'Code_1952': pd.Series(dtype='int')})

In [32]:
for filename in os.scandir(data_dir):
        if ".pdf" in filename.name:
            df = pd.DataFrame({'file_name': pd.Series(dtype='str'),
                                #'court': pd.Series(dtype='str'),                      
                                #'acts': pd.Series(dtype='str'),
                                #'elements_used': pd.Series(dtype='str'),
                                #'location_likely': pd.Series(dtype='str'),
                                #'arresting_department_likely': pd.Series(dtype='float'), 
                                  'Code_1591': pd.Series(dtype='int'), 
                                  'Code_2421': pd.Series(dtype='int'), 
                                  'Code_1594': pd.Series(dtype='int'), 
                                  'Code_2422': pd.Series(dtype='int'), 
                                  'Code_2423': pd.Series(dtype='int'), 
                                  'Code_1328': pd.Series(dtype='int'), 
                                  'Code_1952': pd.Series(dtype='int')})
            '''
            if "Attachment" in filename:
                main_document_name = re.sub("_Attachment[0-9]+", "", filename.name)
                if (cases['file_name'].eq(main_document_name)).any():
                    # TODO (can we scrape the attachments because they are often screenshots)
                    # add the attachment and what it is to the dictionary
                    attachment_type = get_attachment_type()
                else:
                    # TODO
                    # create a new pandas column for the main court file we extracted and add this attachment as the first
                    # in the attachment dict, then 
            else:
            '''
            reader = pypdf.PdfReader(filename.path)
            pypdf2_text = ""
            for i in range(len(reader.pages)):
                pypdf2_text += reader.pages[i].extract_text() 
            df['file_name'] = [filename.name]
            #df['court'] = get_court(pypdf2_text)
            #df['acts'] = get_acts(pypdf2_text)
            #df['elements_used'] = get_elements(pypdf2_text)
            #df['arresting_department_likely'] = get_arresting_dept(pypdf2_text)
            df['Code_1591'] = [get_courtcode(pypdf2_text, "1591")]
            df['Code_2421'] = [get_courtcode(pypdf2_text, "2421")]
            df['Code_1594'] = [get_courtcode(pypdf2_text, "1594")]
            df['Code_2422'] = [get_courtcode(pypdf2_text, "2422")]
            df['Code_2423'] = [get_courtcode(pypdf2_text, "2423")]
            df['Code_1328'] = [get_courtcode(pypdf2_text, "1328")]
            df['Code_1952'] = [get_courtcode(pypdf2_text, "1952")]
            cases = cases.append(df)


C:\Users\Seth\AppData\Local\Temp\ipykernel_328\73401225.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cases = cases.append(df)
C:\Users\Seth\AppData\Local\Temp\ipykernel_328\73401225.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cases = cases.append(df)
C:\Users\Seth\AppData\Local\Temp\ipykernel_328\73401225.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cases = cases.append(df)
C:\Users\Seth\AppData\Local\Temp\ipykernel_328\73401225.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cases = cases.append(df)
C:\Users\Seth\AppData\Local\Temp\ipykernel_328\73401225.py:45: FutureWarning: The frame.append method is dep

KeyboardInterrupt: 

In [ ]:
cases

In [35]:
# messing with pypdf
df = pd.DataFrame({'file_name': pd.Series(dtype='str'),
                    'acts': pd.Series(dtype='str'),
                    'elements_used': pd.Series(dtype='str'),
                    'location_likely': pd.Series(dtype='str'),
                    'arresting_department_likely': pd.Series(dtype='float')})
df['file_name'] = "A.M. v. N.M. Dep't of Health, 148 F. Supp. 3d 1232.pdf"
reader = pypdf.PdfReader("../data/A.M. v. N.M. Dep't of Health, 148 F. Supp. 3d 1232.pdf")
pypdf2_text = ""
for i in range(len(reader.pages)):
    pypdf2_text += reader.pages[i].extract_text() 

FileNotFoundError: [Errno 2] No such file or directory: "./data/A.M. v. N.M. Dep't of Health, 148 F. Supp. 3d 1232.pdf"

In [78]:
print(pypdf2_text)

 Zamora v. Bd. of Educ. for the Las Cruces Pub. Schs
United States District Court for the District of New Mexico
May 22, 2013, Decided; May 22, 2013, Filed
No. 12-CV-550 MCA/CG
Reporter
2013 U.S. Dist. LEXIS 203078 *; 2013 WL 12086304
DENNIS ZAMORA, Plaintiff, v. BOARD OF EDUCATION FOR THE LAS CRUCES PUBLIC SCHOOLS, 
Defendants.
Subsequent History: Affirmed by Zamora v. Bd. of Educ. for the Las Cruces Pub. Sch., 553 Fed. Appx. 786, 2014 
U.S. App. LEXIS 1380 (10th Cir. N.M., Jan. 24, 2014)
Core Terms
Rounds, terminating, employees, investigator, sexual harassment, hostile work environment, proffered, allegations, 
pretext, adverse employment action, summary judgment, national origin, complaints, renew, interviewed, contends, 
argues, hired, discriminatory, Schools, submits, woman, prima facie case, conversation, pretextual, supervised, 
yelled, non discriminatory reason, proffered evidence, deposition
Counsel:  [*1] For Dennis Zamora, Plaintiff: Daniela Labinoti, Law Firm of Daniela La

In [92]:
re.findall('(?<=Core Terms).*$', pypdf2_text)

[]

In [115]:
#pypdf2_text.lower().partition("core terms\n")[2].split('\n')[0:5]
' '.join(pypdf2_text.lower().partition("core terms\n")[2].split('\n')[0:5])

'rounds, terminating, employees, investigator, sexual harassment, hostile work environment, proffered, allegations,  pretext, adverse employment action, summary judgment, national origin, complaints, renew, interviewed, contends,  argues, hired, discriminatory, schools, submits, woman, prima facie case, conversation, pretextual, supervised,  yelled, non discriminatory reason, proffered evidence, deposition counsel:  [*1] for dennis zamora, plaintiff: daniela labinoti, law firm of daniela labinoti pc, el paso, tx usa.'